Scripts to interact with fdp.cmbi.umcn.nl
1. get token
2. get current user
3. reads metadata from TWOC-MultiOmics-Studies-COVID_02DEC2021.xlsx and submit to fdp.cmbi.umcn.nl
4. delete the datasets uploaded
Date: 18 Feb 2022
XiaoFeng Liao

In [40]:
# get token

import requests
import json

#please fill in your username and password
username = "example@example.com"
password = "password"

my_headers = {'content-type':'application/json'}
json_data = json.dumps({
   "email": username,
   "password": password
 })

# make an http-call to obtain a JWT-token
response = requests.post("https://fdp.cmbi.umcn.nl/tokens", json_data, headers=my_headers) 

print(response)
#print(response.json())

# extract JWT access token
data = response.json()
#print(data)
access_token = data["token"]


#print(access_token)

<Response [200]>


In [41]:
# get current user

headers = {
    'accept': 'application/json',
    'Authorization': 'Bearer ' + access_token,
}

response = requests.get('https://fdp.cmbi.umcn.nl/users/current', headers=headers)
response_data = response.json()
print(response)
#print(response_data)
currentUser = response_data['firstName'] + ' ' + response_data['lastName']
#print(response.text)
#print(currentUser)

<Response [200]>


In [45]:
# Submit dataset
# input: TWOC-MultiOmics-Studies-COVID_02DEC2021.xlsx
# tranfer each row from input to a dataset record, submit each record to https://fdp.cmbi.umcn.nl


import pandas


excel_data_df = pandas.read_excel('TWOC-MultiOmics-Studies-COVID_02DEC2021.xlsx', sheet_name='TABLE',skiprows=[1])

#excel_data_df.reset_index()

publisher = currentUser

headers = {'accept': 'text/turtle',
'Authorization': 'Bearer ' + access_token ,
'Content-Type': 'application/ld+json'}


for index, row in excel_data_df.iterrows():
    
    fullmetadata = """ [ {"@id" : "_:genid-c926b452fd0843f0b633c782a6eaa29c-t241", 
    "@type" : [ "http://xmlns.com/foaf/0.1/Agent" ],  
    "http://xmlns.com/foaf/0.1/name" : [ {      
    "@value" :  " """
    fullmetadata += publisher
    fullmetadata +=     """   "   } ] }, 
    {  "@id" : "https://fdp.cmbi.umcn.nl/CovidDataset/c7b31b66-41c4-40c4-93c5-f35e5b393233#accessRights",  
    "@type" : [ "http://purl.org/dc/terms/RightsStatement" ], 
    "http://purl.org/dc/terms/description" : [ { 
     "@value" : "This resource has no access restriction"   } ] }, 
     {  "@id" : "https://fdp.cmbi.umcn.nl/CovidDataset/c7b31b66-41c4-40c4-93c5-f35e5b393233#identifier",  
     "@type" : [ "http://purl.org/spar/datacite/Identifier" ], 
    "http://purl.org/dc/terms/identifier" : [ { 
     "@value" : "https://fdp.cmbi.umcn.nl/CovidDataset/c7b31b66-41c4-40c4-93c5-f35e5b393233"  } ]},     
    {  "@id" : "https://fdp.cmbi.umcn.nl/dataset/2cb82a87-0a08-48c7-b8f8-cb59da64b120",   
    "@type" : [ "http://www.w3.org/ns/dcat#Resource", "http://www.w3.org/ns/dcat#Dataset" ], """

    meta = ''
    for col in row.index:
         if col == 'http://edamontology.org/data_1188':
                meta='"http://edamontology.org/data_1188":[{"@id":"'
                meta += row[col]
                meta += '"}],'
         else:
             #print(type(row[col]))
                #print(col)
                #print(row[col])
                if type(row[col]) is str:
                    meta += '"' + col + '":[{"@value":"' + row[col] + '"}],'
                else:
                    meta += '"' + col + '":[{"@value":"' + str(row[col]) + '"}],'
    fullmetadata += meta
    fullmetadata += """ "http://purl.org/dc/terms/accessRights" : 
    [ { "@id" : "https://fdp.cmbi.umcn.nl/CovidDataset/c7b31b66-41c4-40c4-93c5-f35e5b393233#accessRights"      } ], 
    
    "http://purl.org/dc/terms/conformsTo" : 
    [ {"@id" : "https://fdp.cmbi.umcn.nl/profile/610d5ef8-711c-4290-bfdf-1a20e9833a16" },
    {"@id" : "https://fdp.cmbi.umcn.nl/profile/2f08228e-1789-40f8-84cd-28e3288c3604"      } ], 
    
    "http://purl.org/dc/terms/description" : [ {       "@value" : "IMA1601"      } ], 
    
    "http://purl.org/dc/terms/hasVersion" : [ {       "@value" : "1.0"      } ],
    
    "http://purl.org/dc/terms/isPartOf" : [{ "@id":"https://fdp.cmbi.umcn.nl/catalog/2f807498-1fb7-4ebb-8d1b-2c33d719fb71" }],
    
    "http://purl.org/dc/terms/language" : [ {       "@id" : "http://id.loc.gov/vocabulary/iso639-1/en"      } ],
    
    "http://purl.org/dc/terms/license" : [ {        "@id" : "http://rdflicense.appspot.com/rdflicense/cc-by-nc-nd3.0"}],
    
    "http://purl.org/dc/terms/publisher" : [ {       "@id" : "_:genid-c926b452fd0843f0b633c782a6eaa29c-t241"} ],
    
    "http://purl.org/dc/terms/rights" : [ {       "@id" : "http://rdflicense.appspot.com/rdflicense/cc-by-nc-nd3.0"}], 
    
    "http://purl.org/dc/terms/title" : [ {       "@value" : "IMA1601"} ],
    
    "http://rdf.biosemantics.org/ontologies/fdp-o#metadataIdentifier" : 
    [ { "@id" : "https://fdp.cmbi.umcn.nl/dataset/2cb82a87-0a08-48c7-b8f8-cb59da64b120#identifier" } ], 
    
    "http://rdf.biosemantics.org/ontologies/fdp-o#metadataIssued" : 
    [ { "@type" : "http://www.w3.org/2001/XMLSchema#dateTime",        "@value" : "2022-02-03T20:44:01.699Z"} ], 
    
    "http://rdf.biosemantics.org/ontologies/fdp-o#metadataModified" : 
    [ { "@type" : "http://www.w3.org/2001/XMLSchema#dateTime",        "@value" : "2022-02-03T20:44:01.699Z" } ],   
    
    "http://semanticscience.org/resource/SIO_000628" : 
    [ {"@id" : "https://fdp.cmbi.umcn.nl/dataset/2cb82a87-0a08-48c7-b8f8-cb59da64b120/metrics/445c0a70d1e214e545b261559e2842f4" }, 
    { "@id" : "https://fdp.cmbi.umcn.nl/dataset/2cb82a87-0a08-48c7-b8f8-cb59da64b120/metrics/5d27e854a9e78eb3f663331cd47cdc13" } ],
    
    "http://www.example.com/resources/InvestigationContact" : [ {       "@value" : "Hospital"      } ],  
    
    "http://www.w3.org/2000/01/rdf-schema#label" : [ {"@value" : "IMA1601"} ], 
    
    "http://www.w3.org/ns/dcat#theme" : [ {"@id" : "http://www.wikidata.org/entity/Q14944328"} ] 
    
    }, 
    
    {"@id" : "https://fdp.cmbi.umcn.nl/dataset/2cb82a87-0a08-48c7-b8f8-cb59da64b120#identifier",      
    "@type" : [ "http://purl.org/spar/datacite/Identifier" ],      
    "http://purl.org/dc/terms/identifier" : [ {
    "@value" : "https://fdp.cmbi.umcn.nl/dataset/2cb82a87-0a08-48c7-b8f8-cb59da64b120"} ]       },
    
    { "@id" : "https://fdp.cmbi.umcn.nl/dataset/2cb82a87-0a08-48c7-b8f8-cb59da64b120/metrics/445c0a70d1e214e545b261559e2842f4",      
    "http://semanticscience.org/resource/SIO_000332" : [ { "@id" : "https://www.ietf.org/rfc/rfc3986.txt"      } ],      
    "http://semanticscience.org/resource/SIO_000628" : [ { "@id" : "https://www.ietf.org/rfc/rfc3986.txt"      } ]      },
    
    {"@id" : "https://fdp.cmbi.umcn.nl/dataset/2cb82a87-0a08-48c7-b8f8-cb59da64b120/metrics/5d27e854a9e78eb3f663331cd47cdc13",      
    "http://semanticscience.org/resource/SIO_000332" : [ {       "@id" : "https://www.wikidata.org/wiki/Q8777"      } ],      
    "http://semanticscience.org/resource/SIO_000628" : [ {       "@id" : "https://www.wikidata.org/wiki/Q8777"      } ]  }, 
    
    {"@id" : "https://fdp.cmbi.umcn.nl/distribution/",      
    "@type" : [ "http://www.w3.org/ns/ldp#DirectContainer" ],      
    "http://purl.org/dc/terms/title" : [ { "@value" : "Distribution"      }, {"@value" : "Distributions" } ],      
    "http://www.w3.org/ns/ldp#contains" : [ {"@id" : "https://fdp.cmbi.umcn.nl/distribution/6158b570-89dd-4adf-b96f-b3d496e671fc"} ],      
    "http://www.w3.org/ns/ldp#hasMemberRelation" : [ { "@id" : "http://www.w3.org/ns/dcat#distribution"} ],      
    "http://www.w3.org/ns/ldp#membershipResource" : [ { "@id" : "https://fdp.cmbi.umcn.nl/CovidDataset/c7b31b66-41c4-40c4-93c5-f35e5b393233"}, 
    {"@id" : "https://fdp.cmbi.umcn.nl/dataset/2cb82a87-0a08-48c7-b8f8-cb59da64b120"} ]   },
    
    
    {"@id" : "https://fdp.cmbi.umcn.nl/profile/2f08228e-1789-40f8-84cd-28e3288c3604",      
    "http://www.w3.org/2000/01/rdf-schema#label" : [ {       "@value" : "Dataset Profile" } ]  },
    
    {      "@id" : "https://fdp.cmbi.umcn.nl/profile/610d5ef8-711c-4290-bfdf-1a20e9833a16",      
    "http://www.w3.org/2000/01/rdf-schema#label" : [ {       "@value" : "CovidDataset Profile"      } ]      }
    
    ] """
    #print(fullmetadata)
    

    data = fullmetadata
    response = requests.post('https://fdp.cmbi.umcn.nl/dataset', headers=headers, data=data)
    print(response)
#    print(response.text)
    print('------------------')

    



<Response [201]>
------------------
<Response [201]>
------------------
<Response [201]>
------------------
<Response [201]>
------------------
<Response [201]>
------------------
<Response [201]>
------------------
<Response [201]>
------------------
<Response [201]>
------------------
<Response [201]>
------------------
<Response [201]>
------------------
<Response [201]>
------------------
<Response [201]>
------------------
<Response [201]>
------------------
<Response [201]>
------------------
<Response [201]>
------------------
<Response [201]>
------------------
<Response [201]>
------------------
<Response [201]>
------------------
<Response [201]>
------------------
<Response [201]>
------------------
<Response [201]>
------------------
<Response [201]>
------------------
<Response [201]>
------------------
<Response [201]>
------------------
<Response [201]>
------------------
<Response [201]>
------------------
<Response [201]>
------------------
<Response [201]>
-----------

<Response [201]>
------------------
<Response [201]>
------------------
<Response [201]>
------------------
<Response [201]>
------------------
<Response [201]>
------------------
<Response [201]>
------------------
<Response [201]>
------------------
<Response [201]>
------------------
<Response [201]>
------------------
<Response [201]>
------------------
<Response [201]>
------------------
<Response [201]>
------------------
<Response [201]>
------------------
<Response [201]>
------------------
<Response [201]>
------------------
<Response [201]>
------------------
<Response [201]>
------------------
<Response [201]>
------------------
<Response [201]>
------------------
<Response [201]>
------------------
<Response [201]>
------------------
<Response [201]>
------------------
<Response [201]>
------------------
<Response [201]>
------------------
<Response [201]>
------------------
<Response [201]>
------------------
<Response [201]>
------------------
<Response [201]>
-----------

In [46]:
# publish the dataset


import requests


headers = {
    'accept': 'application/ld+json',
    'Authorization': 'Bearer ' + access_token,
}

response = requests.get('https://fdp.cmbi.umcn.nl/catalog/2f807498-1fb7-4ebb-8d1b-2c33d719fb71', headers=headers)
json_data = json.loads(response.text)
#print(response.text)
#print(type(json_data))
#print(json_data)
#print(type(json_data[2]))
#print(json_data[2])
for dataset in json_data[2]['http://www.w3.org/ns/dcat#dataset']:
#     print(type(dataset))
#     print(dataset['@id'])
    url = dataset['@id']
#    print(type(url))
#    print(url)
#    print()

    json_data = {
        'current': 'PUBLISHED',
    }
    meta_state_url = url+"/meta/state"
    print(meta_state_url)
    print()
    response = requests.put(meta_state_url, headers=headers, json=json_data)
    
    

https://fdp.cmbi.umcn.nl/dataset/841222e4-14ca-4b29-b180-f451a0c12815/meta/state

https://fdp.cmbi.umcn.nl/dataset/605c69ed-fcbf-4d2d-8dea-ce0df4e56649/meta/state

https://fdp.cmbi.umcn.nl/dataset/a9df59f0-4c5c-470a-8199-85b93682f01a/meta/state

https://fdp.cmbi.umcn.nl/dataset/653ebbd9-f0d9-4190-9025-4db94a8c1039/meta/state

https://fdp.cmbi.umcn.nl/dataset/84769bd1-dc06-4979-b5cf-36a3d9ff2e3b/meta/state

https://fdp.cmbi.umcn.nl/dataset/b4eb3db0-4459-4b66-bef1-d2ac5ee00353/meta/state

https://fdp.cmbi.umcn.nl/dataset/933fc04e-1e6d-493e-8c65-007375dacb11/meta/state

https://fdp.cmbi.umcn.nl/dataset/6451bf52-7ffe-48bd-aaea-afbabbbe5cae/meta/state

https://fdp.cmbi.umcn.nl/dataset/002fb1d0-082e-4e84-80c2-0be08d34c516/meta/state

https://fdp.cmbi.umcn.nl/dataset/7ea1cd5e-a491-43dc-8044-7a96b1536cb0/meta/state

https://fdp.cmbi.umcn.nl/dataset/ef45e67c-5039-440b-9b89-d6b33c746a74/meta/state

https://fdp.cmbi.umcn.nl/dataset/3a6e68f9-3843-4063-b8ad-f39065a82547/meta/state

https://fdp.cmbi

https://fdp.cmbi.umcn.nl/dataset/2beea828-43fc-469b-8319-ce47d553efc4/meta/state

https://fdp.cmbi.umcn.nl/dataset/75c5ab79-1e32-456f-b3ed-9f9458b59ef0/meta/state

https://fdp.cmbi.umcn.nl/dataset/951f3f3c-93f7-4dbb-bdbd-e38525e96c21/meta/state

https://fdp.cmbi.umcn.nl/dataset/05469129-6e77-4845-ab80-32f50a150f3a/meta/state

https://fdp.cmbi.umcn.nl/dataset/3f1942f5-ae9a-4abe-ab33-a5632e38599e/meta/state

https://fdp.cmbi.umcn.nl/dataset/db82480b-28ab-4dec-922d-b6103ee2dc15/meta/state

https://fdp.cmbi.umcn.nl/dataset/15a0e1bf-5789-4740-ab2f-6258552a7dfe/meta/state

https://fdp.cmbi.umcn.nl/dataset/775fec86-bb07-482e-8881-e89e128420df/meta/state

https://fdp.cmbi.umcn.nl/dataset/f2c382f5-a7b7-43de-9b46-5f76dc44abd7/meta/state

https://fdp.cmbi.umcn.nl/dataset/b3223844-d205-4296-a838-5af5ba0a3e0f/meta/state

https://fdp.cmbi.umcn.nl/dataset/369b5476-28b3-4848-a42b-4ced84293cb8/meta/state

https://fdp.cmbi.umcn.nl/dataset/21366e93-7005-4ff9-8232-bec01be32b1a/meta/state

https://fdp.cmbi

https://fdp.cmbi.umcn.nl/dataset/27ee70fb-824c-4667-8315-7ee1a8695e91/meta/state

https://fdp.cmbi.umcn.nl/dataset/98c6d55d-5d83-4105-89c1-e81f80123359/meta/state

https://fdp.cmbi.umcn.nl/dataset/3ad1d2ed-2510-44ca-b9bd-aa4f9cc349ed/meta/state

https://fdp.cmbi.umcn.nl/dataset/6f91e283-5308-4915-be2b-29618fae5a08/meta/state

https://fdp.cmbi.umcn.nl/dataset/7b53c07e-992e-4377-ba4b-71e5938aa5b4/meta/state

https://fdp.cmbi.umcn.nl/dataset/b1adcb59-4f81-4ee0-b155-be1d80dd7da7/meta/state

https://fdp.cmbi.umcn.nl/dataset/4aabf399-d09b-44c5-a335-3793717420c1/meta/state

https://fdp.cmbi.umcn.nl/dataset/5edce52b-1100-47d6-9ca1-d690d77c15cc/meta/state

https://fdp.cmbi.umcn.nl/dataset/5d135c15-b732-4024-9b94-47c31faca23a/meta/state

https://fdp.cmbi.umcn.nl/dataset/8e583492-bc35-49b7-b858-dd56320b564f/meta/state

https://fdp.cmbi.umcn.nl/dataset/481535e4-2f84-4483-a8d4-e533bfb2fc93/meta/state

https://fdp.cmbi.umcn.nl/dataset/67c16d19-d566-48b8-8ee7-99487cea7b93/meta/state

https://fdp.cmbi

In [44]:
################## 
# USE WITH CAUTION!
# USE WITH CAUTION!
##################

# 9 Mar,2022
# this cell delete from fdp.cmbi.umcn.nl the datasets which were previously uploaded


headers = {
    'accept': 'application/ld+json',
    'Authorization': 'Bearer ' + access_token,
}

response = requests.get('https://fdp.cmbi.umcn.nl/catalog/2f807498-1fb7-4ebb-8d1b-2c33d719fb71', headers=headers)
json_data = json.loads(response.text)
#print(response.text)
#print(type(json_data))
#print(json_data)
#print(type(json_data[2]))
#print(json_data[2])
for dataset in json_data[2]['http://www.w3.org/ns/dcat#dataset']:
#     print(type(dataset))
#     print(dataset['@id'])
    url = dataset['@id']
#    print(type(url))
    print(url)
    print()

    headers = {
        'accept': '*/*',
        'Authorization': 'Bearer ' + access_token
    }
    response = requests.delete(url, headers=headers)


<class 'list'>
https://fdp.cmbi.umcn.nl/dataset/c8794009-9e64-4747-b9ca-1d0fea985f56

https://fdp.cmbi.umcn.nl/dataset/6aec078f-e10f-4aec-ba08-f9fca4416302

https://fdp.cmbi.umcn.nl/dataset/651bc56b-9492-4713-ac7e-eeaa87916b29

https://fdp.cmbi.umcn.nl/dataset/4d9f27c0-83d2-4997-add3-f8f757f69688

https://fdp.cmbi.umcn.nl/dataset/73cf5ffd-1387-4708-adbd-06f627fd263b

https://fdp.cmbi.umcn.nl/dataset/828565c1-9253-4b44-a4a1-d814edadbb48

https://fdp.cmbi.umcn.nl/dataset/c0261e46-6dfc-4ea9-9128-4e9aa3663006

https://fdp.cmbi.umcn.nl/dataset/b637602c-0497-4a29-9377-ac0f7894dd91

https://fdp.cmbi.umcn.nl/dataset/cad4fd03-e2cb-4630-90e1-7c6375de3fe5

https://fdp.cmbi.umcn.nl/dataset/027a1c89-632f-4115-86e3-9cb395952b94

https://fdp.cmbi.umcn.nl/dataset/6de24dee-7dba-493c-bee5-b5f7973a65bd

https://fdp.cmbi.umcn.nl/dataset/562f2a07-7b1f-430a-a16f-da54ca38dc05

https://fdp.cmbi.umcn.nl/dataset/9471535c-3221-4034-9829-72e2ee688ed2

https://fdp.cmbi.umcn.nl/dataset/42e7d309-6f38-4d7e-a50e-7506e

https://fdp.cmbi.umcn.nl/dataset/e000d057-e19f-4906-a7f9-c614a7fc702f

https://fdp.cmbi.umcn.nl/dataset/7d1e3892-bfac-4f85-a562-c0d1a20f2196

https://fdp.cmbi.umcn.nl/dataset/91d38dd8-d8d2-46d1-b5d5-b6e440a01964

https://fdp.cmbi.umcn.nl/dataset/3e521041-3140-415a-8d0a-b44a858f14f5

https://fdp.cmbi.umcn.nl/dataset/4e843f3d-f635-40ff-9ce3-8adcdb3948e2

https://fdp.cmbi.umcn.nl/dataset/1277ef41-b707-4da0-8e03-f78faa21a204

https://fdp.cmbi.umcn.nl/dataset/6323f02a-7b30-43e1-890f-6b4ee399da49

https://fdp.cmbi.umcn.nl/dataset/4182bd9d-64f1-4006-833b-96772cbbee94

https://fdp.cmbi.umcn.nl/dataset/993d6ed5-a936-4be4-9c48-338278b652a3

https://fdp.cmbi.umcn.nl/dataset/d2a09029-b67d-4496-ab95-020c60868aae

https://fdp.cmbi.umcn.nl/dataset/0ee6caec-7b2c-46d5-9cdf-d6a7b4491f42

https://fdp.cmbi.umcn.nl/dataset/d92674ff-f299-4aa5-a56c-6b4a24ecc57c

https://fdp.cmbi.umcn.nl/dataset/3241535c-6aaf-47e5-8229-c86a60bf6864

https://fdp.cmbi.umcn.nl/dataset/79658c67-6292-4a79-8a21-87dd86911e03

https:

https://fdp.cmbi.umcn.nl/dataset/3778f77c-3f79-427c-842b-691c46a25b94

https://fdp.cmbi.umcn.nl/dataset/b7adc017-acae-470b-8621-c0e1991f6740

https://fdp.cmbi.umcn.nl/dataset/9b9f24ed-0a85-4b46-b61e-a640d3f4b011

https://fdp.cmbi.umcn.nl/dataset/b6f256d8-3af4-46e5-9648-c3251b55d7d4

https://fdp.cmbi.umcn.nl/dataset/5a4dbaea-f50c-4117-8032-c9a36077d9d6

https://fdp.cmbi.umcn.nl/dataset/7353ab94-4563-4726-90e4-ae54d5a2f1dc

https://fdp.cmbi.umcn.nl/dataset/364a7917-aef7-4de8-a555-16d56c440ca1

https://fdp.cmbi.umcn.nl/dataset/db74e4a4-d0f8-43e5-8512-fc4d412c8d57

https://fdp.cmbi.umcn.nl/dataset/872aab4f-e103-4b41-a0ba-2c589ff5b8e1

https://fdp.cmbi.umcn.nl/dataset/0e44f845-c06e-4f71-b39e-56fb322c0156

https://fdp.cmbi.umcn.nl/dataset/743ecd4f-7d49-415e-9a39-4cca11101201

https://fdp.cmbi.umcn.nl/dataset/71afeab6-4aed-4ec7-92a3-8367494cabac

https://fdp.cmbi.umcn.nl/dataset/0a9ab9bc-236e-4d07-a542-7d9b6e5f009b

https://fdp.cmbi.umcn.nl/dataset/397dca94-b4a1-4bfb-8aa0-54b26b4d6d41

https: